## 本笔记本介绍

这是一个使用 4-bit 量化的 [Gemma-2 9b Instruct](https://blog.google/technology/developers/google-gemma-2/) 和 LoRA 适配器的推理笔记本，LoRA 适配器通过我上传的脚本 [这里](https://www.kaggle.com/code/chenxucool/training-gemma-2-9b-4-bit-qlora-fine-tuning) 训练完成。虽然我们可以选择将 LoRA 适配器与基础模型合并以加快推理速度，但盲目这样做可能会引入不可忽略的量化误差。因此，我选择保持 LoRA 适配器未合并。

## 结果

| 子集 | 对数损失 |
| - | - |
| 评估集 | 0.9371 |
| 公共 LB | 0.941 |


## 安装库

和推理部分的库相同，不过这里通过本地路径安装并升级这些库，而不是从 PyPI 下载。这种方式有助于加速安装，尤其是在 Kaggle 比赛或实验中，安装一些较大的包时，可以通过本地预先上传的 `.whl` 文件快速完成。
- **`--no-index`**：表示要从本地文件或指定的路径中查找包，而不是从网络上的 PyPI 获取。
- **`--find-links`**：告诉 `pip` 查找要安装的包时，使用指定的本地路径或文件。
- **`/kaggle/input/lmsys-wheel-files`**：这是一个包含已经编译好的 `wheel` 文件的目录。




In [1]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

Looking in links: /kaggle/input/lmsys-wheel-files
Processing /kaggle/input/lmsys-wheel-files/transformers-4.42.3-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/peft-0.11.1-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/accelerate-0.32.1-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


## 导入包

In [2]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

2024-09-19 12:26:56.805780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 12:26:56.805899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 12:26:56.938967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


确保代码运行时有两个 GPU 可用，否则就中断运行并报错

In [3]:
assert torch.cuda.device_count() == 2

## 配置
该配置将用于加载 Gemma 模型和 LoRA 微调后的模型，并控制模型的输入处理方式，适应不同的推理场景

In [4]:
@dataclass
class Config:
    # 指定 Gemma 模型的目录路径。该路径是模型文件的存储位置，通常在加载预训练模型时使用。
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    
    # 指定 LoRA 微调后的模型权重路径。这是已经经过 LoRA 微调后的模型检查点的存储位置。
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    
    max_length = 2048 # 指定输入文本的最大长度，超过该值，将进行截断。
    batch_size = 4 # 推理时处理的样本批量大小。
    device = torch.device("cuda")  # 使用 CUDA 设备（GPU）进行模型计算。
    
    # 控制是否启用测试时增强（Test Time Augmentation, TTA）。
    # 当启用 TTA 时，可能会交换模型生成的不同响应或对其进行组合，以增强推理的效果。
    # 例如，`<prompt>` 后跟随 `<model-b's response>` 和 `<model-a's response>` 进行推理，以增强模型在推理时的泛化能力。
    tta = False  
    
    # 控制如何应用 max_length 参数。
    # 如果为 True，将把 max_length 平均分配到每个输入段（例如，prompt 和响应），即每个部分的最大长度为 max_length//3。
    # 如果为 False，则对整个拼接后的输入应用 max_length，即所有文本组合起来后长度不得超过 max_length。
    spread_max_length = False 

cfg = Config()

## 加载和预处理数据


In [5]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

对 `test` 数据集中的 `prompt`、`response_a` 和 `response_b` 进行文本预处理，使用 `process_text` 函数解析文本中的 `null` 值，并返回处理后的纯文本数据。处理后的数据会展示前3行，用于验证预处理是否正确。

### 注意事项：
- 使用 `eval` 解析字符串时需要特别小心，因为它会执行传入的表达式。如果数据来源不受信任，可能会带来安全风险。为了安全性，可以考虑使用 `ast.literal_eval` 来解析安全的表达式，而不是直接用 `eval`。
  
  例如：
  ```python
  import ast
  def process_text(text: str) -> str:
      return " ".join(ast.literal_eval(text))
  ```

In [6]:
# 清理带有 null 值的 JSON-like 数据，将其转化为普通字符串。
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

# loc 是 Pandas 中用于选取行或列的方式，这里选择的是整个 prompt 列，并将其重新赋值为处理后的结果。
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(3))

,id,prompt,response_a,response_b
0,136060,"I have three oranges today, I ate an orange ye...",You have two oranges today.,You still have three oranges. Eating an orange...
1,211333,You are a mediator in a heated political debat...,Thank you for sharing the details of the situa...,Mr Reddy and Ms Blue both have valid points in...
2,1233961,How to initialize the classification head when...,When you want to initialize the classification...,To initialize the classification head when per...


## 分词

定义一个名为 `tokenize` 的函数，用于将输入的 `prompt`（提示）、`response_a`（模型 A 的响应）和 `response_b`（模型 B 的响应）进行分词，并生成用于 NLP 模型的 `input_ids` 和 `attention_mask`。函数的设计允许选择性地使用 `spread_max_length` 选项，以不同方式处理和截断文本长度。

In [7]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # 为每个 prompt、response_a 和 response_b 添加对应的标记，明确表明这些文本的来源，让模型能够识别每部分的含义。
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    
    if spread_max_length:
        # 每个 prompt、response_a 和 response_b 将分别分配最大长度的三分之一，即 max_length//3。
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        
        # 组合 input_ids：将每个 prompt、response_a 和 response_b 的分词结果拼接起来，生成完整的输入序列。
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        
        # 通过生成与 input_ids 等长的 1 列表，表示这些 token 都是有效的（1 表示非 padding 的 token）。
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        # 直接将 prompt、response_a 和 response_b 拼接成一个完整的文本序列，然后作为一个整体进行分词处理。
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        
        # 分词器返回的 input_ids 是分词后的 token 序列，而 attention_mask 用于标记哪些 token 是有效的（1），哪些是 padding token（0）。
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

In [8]:
%%time

# 加载预训练的 Gemma 分词器
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
# 在每个输入文本的末尾添加 <eos>（End of Sequence）标记。这通常用于生成任务，告诉模型输入序列的结束。
tokenizer.add_eos_token = True
# 设置填充方向为右侧。即如果输入文本长度不足最大长度，将在右侧填充 <pad> 标记。
tokenizer.padding_side = "right"

# 分词
# 调用之前定义的 tokenize 函数，对 test 数据集中的 prompt、response_a 和 response_b 进行分词。
# 生成的 input_ids（token 序列）和 attention_mask（注意力掩码）分别存储在 data 的相应列中。
data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

# 数据增强
# 再一次调用 tokenize 函数，不同的是，这次将 response_a 和 response_b 交换，生成新的 input_ids 和 attention_mask。
# 这种方法称为 测试时增强（TTA），即通过交换模型生成的不同响应来增加预测的多样性，帮助提升模型的泛化能力。
aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)

CPU times: user 768 ms, sys: 129 ms, total: 897 ms
Wall time: 1.07 s


查看 data 数据框中第一个样本的 input_ids 所对应的文本，验证分词和拼接后的文本格式是否符合预期。

In [9]:
# tokenizer.decode()：这是 Hugging Face tokenizer 提供的方法，用于将模型的 token IDs 转换回原始的可读文本。
print(tokenizer.decode(data["input_ids"][0]))

<bos><prompt>: I have three oranges today, I ate an orange yesterday. How many oranges do I have?

<response_a>: You have two oranges today.

<response_b>: You still have three oranges. Eating an orange yesterday does not affect the number of oranges you have today.<eos>


In [10]:
print(tokenizer.decode(aug_data["input_ids"][0]))

<bos><prompt>: I have three oranges today, I ate an orange yesterday. How many oranges do I have?

<response_a>: You still have three oranges. Eating an orange yesterday does not affect the number of oranges you have today.

<response_b>: You have two oranges today.<eos>


## 加载模型
将相同的模型分别加载到 GPU 0 和 GPU 1 上，以便在多 GPU 环境下加速模型的计算，并行进行推理。

In [11]:
# 加载基础模型到 GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False, # 禁用缓存（通常用于生成任务时启用）。对于分类任务，禁用缓存不会影响性能，并且可能节省内存。
)

# 加载基础模型到 GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 加载 LoRA 适配器

In [12]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)

## 推理

该推理函数的目标是对给定的数据框（`df`）中的样本进行推理，生成模型A获胜、模型B获胜以及平局的概率，并将这些概率结果添加到原始数据框中。


In [13]:
@torch.no_grad() # 在推理阶段不需要计算梯度
@torch.cuda.amp.autocast() # 这是自动混合精度（AMP）模式，用于在推理时自动使用半精度浮点数（FP16），从而减少显存占用并加快推理速度。通常用于 GPU 上的推理。
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], [] # 分别用于存储模型A获胜、模型B获胜以及平局的概率值。
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        
        # 这个函数的作用是对输入进行填充。填充方式是根据输入序列的最长序列进行填充，以确保批处理中的所有样本长度一致。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt", # 表示将结果转换为 PyTorch 张量（tensor）。
        )
        
        # 将填充后的输入数据移到指定的设备（如 GPU），并通过模型进行推理，生成 logits（模型的输出）。
        outputs = model(**inputs.to(device))
        
        # 对 logits 进行 softmax 操作，将模型的输出转换为概率分布。
        # -1 表示在最后一个维度上进行 softmax，即每个样本的类别概率。
        # 将输出的张量移动到 CPU，便于后续操作。
        proba = outputs.logits.softmax(-1).cpu()
        
        # 存储推理结果
        a_win.extend(proba[:, 0].tolist()) # 提取第一个类别（模型A获胜）的概率并存储到 a_win 列表中。
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    # 更新原始的df
    df["winner_model_a"] = a_win # 将 a_win 列表中的概率值赋给df的 winner_model_a 列，表示模型A获胜的概率。
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df

使用两个不同的 GPU 同时运行模型推理，并借助 Python 的 `ThreadPoolExecutor` 来并行化处理。通过按输入序列长度排序数据，充分利用动态填充（dynamic padding）的优化，并将数据划分为两个子集 `sub_1` 和 `sub_2`，分别在两个 GPU 上运行推理任务，最后合并结果。

In [14]:
st = time.time()

# 将数据框 data 按照 length 列（表示输入序列的长度）降序排序。
# 这样做的目的是为了充分利用动态填充（dynamic padding），即在一个批次内，所有序列都会填充到批次中最长的序列长度。通过这种方式，减少填充冗余，提高推理的效率。
data = data.sort_values("length", ascending=False)

# 将 data 分成两个子集，这种方式确保两个子集的大小大致相同，且每个子集中的序列长度分布相似。这样可以平衡两个 GPU 上的工作负载。
sub_1 = data.iloc[0::2].copy() # 从 data 中每隔一行取一行（即索引为 0, 2, 4,... 的行）。
sub_2 = data.iloc[1::2].copy() # 从 data 中每隔一行取一行，但从索引 1 开始（即索引为 1, 3, 5,... 的行）。

# 通过两个线程分别处理 sub_1 和 sub_2，充分利用两个 GPU 的计算资源实现并行推理。
with ThreadPoolExecutor(max_workers=2) as executor:
    # 将推理任务并行地映射到不同的数据子集、模型和设备上
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

# 按行合并结果
result_df = pd.concat(list(results), axis=0)

# 提取概率，这些概率可以用于后续的决策或评估，比如判断哪个模型的表现更好。
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")

elapsed time: 4.52712869644165


实现测试时增强（**TTA**），并使用两个不同的 GPU 并行运行推理。主要目的是通过交换模型A和模型B的响应来进行 TTA，并最终将原始推理结果与 TTA 结果进行平均。通过这种方式，可以提升模型在测试阶段的泛化能力，进一步提高推理结果的鲁棒性。

In [15]:
st = time.time()

if cfg.tta: # 检查是否启用 TTA
    # 排序增强数据集 aug_data
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    # 并行执行推理
    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    # 合并 TTA 结果
    tta_result_df = pd.concat(list(results), axis=0)
    
    # 交换模型A和模型B的结果顺序
    # TTA 的顺序是颠倒的：因为在 TTA 中，response_a 和 response_b 被交换，因此 TTA 结果中模型A和模型B的概率需要重新排序
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    
    # 计算平均概率，通过结合原始结果和 TTA 结果，提升模型的稳定性和泛化能力。
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")

elapsed time: 0.00018978118896484375


将推理结果格式化并保存为 CSV 文件，用于提交或者进一步处理。

In [16]:
result_df.loc[:, "winner_model_a"] = proba[:, 0] # 提取第一列，即模型A获胜的概率。
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)

,id,winner_model_a,winner_model_b,winner_tie
2,1233961,0.146055,0.644424,0.209521
0,136060,0.006395,0.962159,0.031446
1,211333,0.340726,0.243892,0.415382
